Overview
The data has been split into two groups:

training set (train.csv)
test set (test.csv)
The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

Data Dictionary
Variable	Definition	Key
survival	Survival	0 = No, 1 = Yes
pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
sex	Sex	
Age	Age in years	
sibsp	# of siblings / spouses aboard the Titanic	
parch	# of parents / children aboard the Titanic	
ticket	Ticket number	
fare	Passenger fare	
cabin	Cabin number	
embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton
Variable Notes
pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.

https://www.kaggle.com/c/titanic/overview


In [3]:
#Main Code

import pandas as pd      
import matplotlib as mat
import matplotlib.pyplot as plt    
import numpy as np                
%matplotlib inline  
import sklearn as sk
from sklearn import metrics
#importing Algorithms to be used
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeRegressor


#creating algorithms
model_v1 = GaussianNB()
model_v2 = RandomForestClassifier(random_state = 70)
model_v3 = LogisticRegression(C = 0.7, random_state = 70, max_iter = 10000, solver='newton-cg')
model_v4 = LinearSVC(random_state=1)
model_v5 = DecisionTreeRegressor(random_state=1)

def replaceMissingMean(df, column):    
    df = df
    column = column
    df[column].fillna(int(df[column].mean()),inplace=True)
    return df

def replaceMissingZero(df, column):    
    df = df
    column = column
    df[column].fillna('Vazio',inplace=True)
    return df

def replaceSextoNumbers(df):
    train_test_data = df
    for dataset in train_test_data:
        dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
    
    
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns
    

# Identificando a correlação entre as variáveis
# Correlação não implica causalidade
def plot_corr(df, size=10):
    corr = df.corr()    
    fig, ax = plt.subplots(figsize = (size, size))
    ax.matshow(corr)  
    plt.xticks(range(len(corr.columns)), corr.columns) 
    plt.yticks(range(len(corr.columns)), corr.columns)  

def predict(array,passengerId,model):
    import pickle
    loaded_model = model
    array = array
    passengerId = passengerId
    filename = 'model_v3.sav'
   
    loaded_model = pickle.load(open(filename, 'rb'))
    resultado1 = loaded_model.predict(array[passengerId].reshape(1, -1))
    return resultado1
    

def slicingDfTrain(X,Y,splitsize):
    X = X
    Y = Y
    from sklearn.model_selection import train_test_split
    split_test_size =  splitsize
    # Criando dados de treino e de teste
    X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = split_test_size, random_state = 42)
    # Imprimindo os resultados
    print("{0:0.2f}% nos dados de treino".format((len(X_treino)/(len(X_treino) + len(X_teste))) * 100))
    print("{0:0.2f}% nos dados de teste".format((len(X_teste)/(len(X_treino) + len(X_teste))) * 100))
    return [X_treino, X_teste, Y_treino, Y_teste]

def saveAlgorithm(model):
    model_v3=model
    # saving algorithm for later use
    import pickle
    filename = 'model_v3.sav'
    pickle.dump(model_v3, open(filename, 'wb'))
    print('Algorithm SAVED with Sucess!! \n %s' %(model_v3))

def createOutputFile(df):
    df = df
    df.to_csv('testou.csv')
    df.to_excel("output.xlsx",sheet_name='Sheet_name_1')

In [4]:
#Looking for some correlation in variables
plot_corr(dfTrain)

NameError: name 'dfTrain' is not defined

In [2]:
dfTrain

NameError: name 'dfTrain' is not defined

In [5]:
#creating DF to put features and outputs
dfTrain = pd.read_csv('train.csv')
dfTrain = replaceMissingMean(dfTrain,'Age')

dfPredict = pd.read_csv('test.csv')

#Setting features
features = ['PassengerId', 'Pclass', 'Age', 'SibSp','Parch', 'Fare']
XTrain = dfTrain[features].values

#Setting output
YTrain = dfTrain['Survived'].values


#Split TRAIN database to test model later
X_treino, X_teste, Y_treino, Y_teste = slicingDfTrain(XTrain,YTrain,0.3)

#Tryning models (Machine Learning Algorithms)
models = [ model_v1, model_v2, model_v3, model_v4,model_v5]
indices = [0,1,2,3,4]

algorithmRates = []
# Chosing best algorithm
for i in indices:
    model = models[i]
    model.fit(X_treino, Y_treino.ravel())
    nb_predict_train = model.predict(X_teste)

    print("Model selected",model, "Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, nb_predict_train) ))
    algorithmRates.append(metrics.accuracy_score(Y_teste, nb_predict_train))
    print()
    
saveAlgorithm(model_v3)    

69.92% nos dados de treino
30.08% nos dados de teste
Model selected GaussianNB(priors=None, var_smoothing=1e-09) Exatidão (Accuracy): 0.6940

Model selected RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=70, verbose=0,
                       warm_start=False) Exatidão (Accuracy): 0.7201

Model selected LogisticRegression(C=0.7, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=70, solv

C:\Users\hansf\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [83]:
dfTrain.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,29.0,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [149]:
dfTrain['Cabin#'] = dfTrain['Cabin']
count = 0
for i in dfTrain['Cabin#']:
    if dfTrain['Cabin?'][count]!='Vazio':
        for i in dfTrain['Cabin#'][count]:
            letra = i
            dfTrain['Cabin#'][count] = i
            break
        #print('Cabin number', count ,dfTrain['Cabin?'][count])
    count = count + 1    

C:\Users\hansf\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [150]:
dfTrain

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family Size,Cabin?,Cabin#
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Vazio,S,2,Vazio,Vazio
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Vazio,S,1,Vazio,Vazio
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,C,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Vazio,S,1,Vazio,Vazio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,Vazio,S,1,Vazio,Vazio
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,B,B
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.0,1,2,W./C. 6607,23.4500,Vazio,S,6,Vazio,Vazio
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1,C,C


In [151]:
count = 0
listaProv = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8}

for i in dfTrain['Cabin#']:
    if dfTrain['Cabin#'][count] == i:
        dfTrain['Cabin#'][count] = listaProv.get(i)
    else:
        dfTrain['Cabin#'][count] = 9
    count = count + 1


C:\Users\hansf\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [152]:
dfTrain

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family Size,Cabin?,Cabin#
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Vazio,S,2,Vazio,None
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Vazio,S,1,Vazio,None
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,C,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Vazio,S,1,Vazio,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,Vazio,S,1,Vazio,None
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,B,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.0,1,2,W./C. 6607,23.4500,Vazio,S,6,Vazio,None
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1,C,2


In [125]:
replaceMissingZero(dfTrain, 'Cabin')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family Size,Cabin?,Cabin#
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Vazio,S,2,Vazio,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Vazio,S,1,Vazio,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,C,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Vazio,S,1,Vazio,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,Vazio,S,1,Vazio,2
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,B,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.0,1,2,W./C. 6607,23.4500,Vazio,S,6,Vazio,2
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1,C,2


In [63]:
#Creating submission Df
dfFinal = pd.DataFrame(dfPredict, columns=['PassengerId'])
dfFinal['Survived'] = 0

#check for missing values + Correcting
replaceMissingMean(dfPredict,'Age')
replaceMissingMean(dfPredict,'Fare')
missing_values_table(dfPredict)

# Predicting values
features = ['PassengerId', 'Pclass', 'Age', 'SibSp','Parch', 'Fare']
X_predict = dfPredict[features].values
# X_predict[0]

# Continuing predicting and saving in DfFinal
count = 0
model = model_v3
lista = []
for i in X_predict:
    variable =  predict(X_predict,count,model)
    dfFinal['Survived'][count] = variable
    count = count + 1
print(dfFinal) 

#savinf in excel file
dfFinal = dfFinal.set_index('PassengerId')
createOutputFile(dfFinal)

print('File saved!!')


Your selected dataframe has 11 columns.
There are 1 columns that have missing values.
     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
File saved!!


In [60]:
print('Total passengers: %s \n' %( dfFinal.Survived.count()))
print('Total passengers survided: %s' %( dfFinal.Survived.sum()))
print('                          {0:.2f}%'.format(  (dfFinal.Survived.sum()/dfFinal.Survived.count())*100))
print('Average Sucess Rate: {0:.2f}% \n'.format(max(algorithmRates)*100))

Total passengers: 418 

Total passengers survided: 94
                          22.49%
Average Sucess Rate: 72.39% 



In [62]:
dfFinal.set_index('PassengerId')

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,0
...,...
1305,0
1306,1
1307,0


In [71]:
createOutputFile(dfFinal)